In [1]:
import gcapi  # pip install gcapi

your_algorithm_slug = "nnunet_segmentation_for_detection"           # <--- CHANGE THIS
client = gcapi.Client(token="0efc24961ab263566ba64ff87579a2079cffc932b366a977b96976f60ca4a385")                           # <--- CHANGE THIS, more information about the token please see this link: https://grand-challenge.org/documentation/what-can-gc-api-be-used-for/

job = client.run_external_job(
    algorithm=your_algorithm_slug,
    inputs={
        # 104S (from the tils training subset)
        "generic-medical-image": "https://grand-challenge.org/api/v1/cases/images/18a9e579-34bd-43b7-ac42-61541fb35156/",
        # 104S_rois (similar rois as expected in L1)  
        "generic-overlay": "https://grand-challenge.org/api/v1/cases/images/e676fdaa-719e-4050-81b7-4724fed69c52/"
    }
)
# More information about gcapi please see this link: https://grand-challenge.org/documentation/grand-challenge-api/

In [20]:
print(job["status"] )

Queued


In [21]:
job

{'pk': '9876a599-5656-4c37-a398-1f37c87d85ec',
 'inputs': [{'interface': 'generic-medical-image',
   'value': None,
   'file': None,
   'image': 'https://grand-challenge.org/api/v1/cases/images/18a9e579-34bd-43b7-ac42-61541fb35156/',
   'pk': 1919147},
  {'interface': 'generic-overlay',
   'value': None,
   'file': None,
   'image': 'https://grand-challenge.org/api/v1/cases/images/e676fdaa-719e-4050-81b7-4724fed69c52/',
   'pk': 1919148}],
 'status': 'Queued'}

In [22]:
job2 = client.algorithm_jobs.detail(job["pk"])

In [23]:
job2

{'pk': '9876a599-5656-4c37-a398-1f37c87d85ec',
 'url': 'https://grand-challenge.org/algorithms/nnunet_segmentation_for_detection/jobs/9876a599-5656-4c37-a398-1f37c87d85ec/',
 'api_url': 'https://grand-challenge.org/api/v1/algorithms/jobs/9876a599-5656-4c37-a398-1f37c87d85ec/',
 'algorithm_image': 'https://grand-challenge.org/api/v1/algorithms/images/c0966089-24c3-4e3e-890f-920bb51492d6/',
 'inputs': [{'interface': {'title': 'Generic Medical Image',
    'description': '',
    'slug': 'generic-medical-image',
    'kind': 'Image',
    'pk': 1,
    'default_value': None,
    'super_kind': 'Image',
    'relative_path': '',
    'overlay_segments': [],
    'look_up_table': None},
   'value': None,
   'file': None,
   'image': 'https://grand-challenge.org/api/v1/cases/images/18a9e579-34bd-43b7-ac42-61541fb35156/',
   'pk': 1919147},
  {'interface': {'title': 'Generic Overlay',
    'description': 'An overlay of unknown type. Legacy, please use alternative interfaces.',
    'slug': 'generic-over

In [24]:
print(job2["status"] )

Failed


In [ ]:
output = job["outputs"]
print(output["image"])

In [ ]:
import gcapi
from pathlib import Path
from tqdm import tqdm
import SimpleITK as sitk
import numpy as np
import os

# loop through job outputs 
for output in job["outputs"]:

    # check whether if output exists
    if output["image"] is not None:

        # get image details
        image_details = client(url=output["image"])

        for file in image_details["files"]:

            # create the output filename
            output_file = Path(input_fname.replace(".mha", "_lobes.mha"))
            if output_file.suffix != ".mha":
                raise ValueError("Output file needs to have .mha extension")
            output_file.parent.mkdir(parents=True, exist_ok=True)

            with output_file.open("wb") as fp:
                # get the image from url and write it 
                response = client(url = file["file"],follow_redirects=True).content
                fp.write(response)
